In [1]:
import pandas as pd
import numpy as np

1. The top 5 polutants (globally)
2. Most polluted locations (by Country/City)
    - in terms of number of pollutants 
    - in terms of concentration
3. Getting the population in locations where possible

In [2]:
dfmain = pd.read_csv('source//temp//AIRclean.csv')
dfmain.shape

(43450, 11)

In [34]:
#dfmain[dfmain['Value']<0].head()
dfmain.describe()

,Value,Lat,Long
count,4.345000e+04,43231.000000,43231.000000
mean,7.655028e+02,32.746883,26.509411
std,3.270964e+04,20.097023,77.983076
min,-4.905140e+04,-74.166278,-161.767000
25%,1.900000e+00,27.803900,-2.940556
50%,1.060000e+01,37.784278,20.212689
75%,3.500000e+01,44.849695,105.807650
max,3.198676e+06,78.906690,172.681343


Minimum in Value is quite large negative number. I would expect to have minimum as zero accross pollutant.

In [35]:
dfmain[dfmain['Value']<0].head()

,Country Code,Country Label,City,Location,Pollutant,Source Name,Unit,Value,Last Updated,Lat,Long
391,CA,Canada,BRITISH COLUMBIA,FORT ST JOHN LEARNIN,NO,AirNow,ppm,-0.0001,2023-08-09T11:00:00+04:00,56.244722,-120.856111
412,AE,United Arab Emirates,Dubai,US Diplomatic Post: Dubai,PM2.5,StateAir_Dubai,µg/m³,-999.0000,2023-08-09T12:00:00+04:00,25.258480,55.309166
487,CO,Colombia,Medellin,SUR-TRAF - Estación Tráfico Sur,PM10,Medellin,µg/m³,-9999.0000,2021-09-30T17:00:00+04:00,6.152313,-75.627487
491,CO,Colombia,NaN,GIR-IECO - Institucion Educativa Colombia. Gir...,PM10,Medellin,µg/m³,-9999.0000,2021-09-30T17:00:00+04:00,6.378361,-75.443886
617,SE,Sweden,Stockholm,Lilla Essingen (E4/E20),NO2,Stockholm,µg/m³,-99.0000,2020-12-11T18:00:00+04:00,59.325519,18.003961


In [36]:
dfmain[dfmain['Value']<0].shape

(595, 11)

In [39]:
# So having some doubts that negative values in polutants is reliable measure, I will drop those. 
# Especially given that it's only ~600 out of 43k - we most likely won't loose much of information for the overview.
dfmain = dfmain[dfmain['Value']>=0]

In [7]:
# While it is not exactly descriptive, it's still interesting to know the top 5 polluters globally (e.g. we can track the source further to find relations etc)
dfmain['Pollutant'].drop_duplicates().size

10

Below we are getting a slice of DF where pollutant grouped by the name and units (mean and max aggregating function). And we also display a country name with the Max level of the corresponding pollutant.

In [28]:
'''
max_merged = country_max.groupby(['Pollutant', 'Unit']).apply(
    lambda x: pd.Series({
        'Country Label with Max Value': x['Country Label'][x['max'].idxmax()],
        'Max Value': x['max'].max(),
        'Mean Value': x['mean'][x['max'].idxmax()]
    })
).reset_index()
max_merged.head(5)
'''

"\nmax_merged = country_max.groupby(['Pollutant', 'Unit']).apply(\n    lambda x: pd.Series({\n        'Country Label with Max Value': x['Country Label'][x['max'].idxmax()],\n        'Max Value': x['max'].max(),\n        'Mean Value': x['mean'][x['max'].idxmax()]\n    })\n).reset_index()\nmax_merged.head(5)\n"

In [40]:
df_pol = dfmain.groupby(['Pollutant', 'Unit'])['Value'].agg(['mean', 'max']).reset_index() # max pollutant globally, gruoped by measure units
country_max = dfmain.groupby(['Pollutant', 'Unit', 'Country Label'])['Value'].max().reset_index() # max level per country per pollutant
# it's basically the same as df_pol, but we also identify country name with the max pollutant 
max_merged = country_max.groupby(['Pollutant', 'Unit']).apply(lambda x: x[x['Value'] == x['Value'].max()]['Country Label'].iloc[0]).reset_index(name='Country Label with Max Value')

# now we need to merge two data sets, to have country name and actual level of pollutant // mean and max are on global level
df_pol = df_pol.merge(max_merged, on=['Pollutant', 'Unit'])

In [41]:
print(df_pol)

   Pollutant   Unit         mean           max Country Label with Max Value
0         BC  µg/m³     0.596927  7.408370e+00                       Poland
1         CO    ppm     3.322073  1.242358e+03                       Mexico
2         CO  µg/m³  9529.692726  3.198676e+06                       Turkey
3         NO    ppm     0.007404  1.711500e+00                       Mexico
4         NO  µg/m³     6.725494  8.051200e+02                      Iceland
5        NO2    ppm     0.188879  7.228910e+01                      Ecuador
6        NO2  µg/m³    20.042729  1.010644e+03                       Turkey
7        NOX    ppm     0.014790  1.714300e+00                       Mexico
8        NOX  µg/m³    13.733728  6.700000e+01                        Italy
9         O3    ppm     0.035444  6.670000e-01                     Thailand
10        O3  µg/m³    77.647063  2.004076e+03                       Turkey
11       PM1  µg/m³     2.262500  4.000000e+00                        Spain
12      PM10

Since we are devided between measure units, and there is no reliable way to convert one to another (without additional input, e.g. tempreture, pressure etc at the particulare point of time in the particular place). 
We try to actually see how dominant those measures accross the dataset.

In [66]:
# Counting Units per pollutant
tot_per_pol = dfmain.groupby('Pollutant').size()  # total pollutant observation entries per polutant
df_unit_count = dfmain.groupby(['Pollutant', 'Unit']).size().reset_index(name="Unit Count")
df_unit_count['% of unit'] = df_unit_count.apply(lambda row: round((row['Unit Count']/ tot_per_pol[row['Pollutant']])*100,2), axis=1)
piv_res = df_unit_count.pivot(index='Pollutant', columns = 'Unit', values = '% of unit').reset_index()
piv_res = piv_res.rename(columns={'ppm': 'ppm units %', 'µg/m³': 'µg/m³ units % '})
piv_res

Unit,Pollutant,ppm units %,µg/m³ units %
0,BC,NaN,100.00
1,CO,24.19,75.81
2,NO,26.62,73.38
3,NO2,19.44,80.56
4,NOX,79.46,20.54
5,O3,36.73,63.27
6,PM1,NaN,100.00
7,PM10,NaN,100.00
8,PM2.5,0.08,99.92
9,SO2,25.59,74.41


So the next would be reasonable to identify the same as above not only per pollutant but also per country. 
And then to identify which unit measure is actually appropriate to take to which country. E.g. we can see that Germany is high by SO2, so as Ecuador.
But maybe it doesn't make sence to represent Ecuador for SO2 at all, since they are not measuring in this units much, or proportionaly it's very small.
We also need slices and ratings per each country.